<a href="https://colab.research.google.com/github/shiyadong123/hello-world/blob/master/%E5%88%A9%E7%94%A8RNN_LSTM%E8%BF%9B%E8%A1%8C%E6%83%85%E6%84%9F%E5%88%86%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 利用RNN-LSTM进行情感分析

# 1. RNN简介

## 1.1 RNN模型

循环神经网络（Recurrent Neural Network，RNN）是用来建模序列化数据的一 种主流深度学习模型。我们知道，传统的前馈神经网络一般的输入都是一个定长 的向量，无法处理变长的序列信息，即使通过一些方法把序列处理成定长的向量，模型也很难捕捉序列中的长距离依赖关系。RNN则通过将神经元串行起来处理序列化的数据。由于每个神经元能用它的内部变量保存之前输入的序列信息， 因此整个序列被浓缩成抽象的表示，并可以据此进行分类或生成新的序列。近年来，得益于计算能力的大幅提升和模型的改进，RNN在很多领域取得了突破性的进展——机器翻译、序列标注、图像描述、推荐系统、智能聊天机器人、自动作词作曲等。<br>

下图即为一个典型的RNN网络结构：<br>
一个长度为$T$的序列用RNN建模，展开之后可以看作是一个$T$层的前馈神经网络。其中，第$t$层的隐含状态$h_t$编码了序列中前$t$个输入的信息，可以通过当前的输入$x_t$和上一层神经网络的状态$h_{T-1}$计算得到；最后一层的状态$h_{T}$编码了整个序列的信息。


![png](https://github.com/shiyadong123/Myimage/blob/master/RNN00.PNG?raw=true)

## 1.2 LSTM简介

+ RNN模型的缺陷

RNN模型的求解采用BPTT（Back Propagation Through Time，基于时间的反向传播）算法实现，BPTT实际上是反向传播算法的简单变种。如果将RNN按时间展开成T层的前馈神经网络来理解，就和普通的反向传播算法非常类似。然而实践发现，使用BPTT算法学习的RNN模型并不能成功捕捉到长距离的依赖关系，这一现象主要源于深度神经网络中的**梯度消失**。<br>

对于普通的前馈网络来说，梯度消失意味着无法通过加深网络层次来改善神经网络的预测效果，因为无论如何加深网络，*只有靠近输出的若干层才真正起到学习的作用*，这使得RNN模型很难学习到输入序列中的长距离依赖关系。<br>

而LSTM模型通过加入**门控机制**，很大程度上弥补了梯度消失所带来的损失。


![jpq](https://github.com/shiyadong123/Myimage/blob/master/lstm03.jpg?raw=true)

+ LSTM模型与一般RNN模型的比较

如上图所示，左图是一般的RNN模型，右图是LSTM模型。<br>
普通RNN模型，只是接收上一步的输出和当前步的输入，通过tanh激活函数，获得当前输出并传递到下一步。<br>
而在LSTM模型中，每个$\sigma$都是一个乘法门，决定输入的特征是否重要。

+ LSTM模型的结构

如下图所示，红框从左至右分别是**遗忘门**、**输入门**、**输出门**<br>
遗忘门：决定从cell状态中丢弃（或遗忘）什么信息，通过当前时刻的输入与前一个时刻的输出决定<br>
输入门：确定并更新新信息到当前时刻的cell状态中<br>
输出门：基于目前的cell状态决定该时刻的输出

![png](https://github.com/shiyadong123/Myimage/blob/master/lstm04.png?raw=true)

In [0]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('./Tweets.csv',usecols = ['text','airline_sentiment'])
data.head()

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...


In [3]:
# 使用“0”“1”“2”对应“neutral”“positive”“negative”
data.airline_sentiment = data.airline_sentiment.map({'neutral':0, 'positive':1,'negative':2})
data.head()

,airline_sentiment,text
0,0,@VirginAmerica What @dhepburn said.
1,1,@VirginAmerica plus you've added commercials t...
2,0,@VirginAmerica I didn't today... Must mean I n...
3,2,@VirginAmerica it's really aggressive to blast...
4,2,@VirginAmerica and it's a really big bad thing...


In [4]:
# 对评论做基本清洗，包括转换成小写、去掉特殊符号、分词
import re
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
data['text'] = data['text'].apply(lambda x: x.split())
data.head()

,airline_sentiment,text
0,0,"[virginamerica, what, dhepburn, said]"
1,1,"[virginamerica, plus, youve, added, commercial..."
2,0,"[virginamerica, i, didnt, today, must, mean, i..."
3,2,"[virginamerica, its, really, aggressive, to, b..."
4,2,"[virginamerica, and, its, a, really, big, bad,..."


In [5]:
data['length'] = data['text'].apply(len)
data.head()

,airline_sentiment,text,length
0,0,"[virginamerica, what, dhepburn, said]",4
1,1,"[virginamerica, plus, youve, added, commercial...",9
2,0,"[virginamerica, i, didnt, today, must, mean, i...",12
3,2,"[virginamerica, its, really, aggressive, to, b...",17
4,2,"[virginamerica, and, its, a, really, big, bad,...",10


In [6]:
data.length.describe()

count    14640.000000
mean        17.500068
std          6.871166
min          2.000000
25%         12.000000
50%         19.000000
75%         23.000000
max         35.000000
Name: length, dtype: float64

In [0]:
# 将data中的评论全部合并到同一个list中，便于下面操作
text = data['text'].sum()

In [0]:
from collections import Counter
# 统计text中词出现的频率 
counts = Counter(text)
# 按照词出现的频率进行降序排列，并生成一个单词到索引的字典
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {word: ii for ii, word in enumerate(vocab, 1)}

In [9]:
vocab_to_int

{'to': 1,
 'the': 2,
 'i': 3,
 'a': 4,
 'united': 5,
 'you': 6,
 'for': 7,
 'flight': 8,
 'on': 9,
 'and': 10,
 'my': 11,
 'usairways': 12,
 'americanair': 13,
 'is': 14,
 'in': 15,
 'southwestair': 16,
 'jetblue': 17,
 'of': 18,
 'me': 19,
 'it': 20,
 'your': 21,
 'have': 22,
 'was': 23,
 'not': 24,
 'with': 25,
 'that': 26,
 'at': 27,
 'no': 28,
 'this': 29,
 'get': 30,
 'but': 31,
 'be': 32,
 'from': 33,
 'can': 34,
 'are': 35,
 'thanks': 36,
 'cancelled': 37,
 'we': 38,
 'now': 39,
 'an': 40,
 'just': 41,
 'service': 42,
 'do': 43,
 'so': 44,
 'been': 45,
 'help': 46,
 'time': 47,
 'im': 48,
 'will': 49,
 'customer': 50,
 'up': 51,
 'out': 52,
 'our': 53,
 'they': 54,
 'us': 55,
 'hours': 56,
 'what': 57,
 'when': 58,
 'flights': 59,
 '2': 60,
 'amp': 61,
 'hold': 62,
 'how': 63,
 'its': 64,
 'if': 65,
 'plane': 66,
 'all': 67,
 'thank': 68,
 'why': 69,
 'cant': 70,
 'there': 71,
 'still': 72,
 'please': 73,
 'one': 74,
 'need': 75,
 'would': 76,
 'delayed': 77,
 'virginamerica': 7

In [10]:
# 提取出每一条评论，将评论的每一个word转换成vocab_to_int字典中对应的int索引，即生成词向量
reviews_ints = []
for review in data['text']:
    reviews_ints.append([vocab_to_int[word] for word in review])

# 字典中word数量
print('Unique words: ', len((vocab_to_int)))
# 展示一条评论被转换成词向量的效果
print('Tokenized review: \n', reviews_ints[:1])

Unique words:  16722
Tokenized review: 
 [[78, 57, 6612, 218]]


In [11]:
reviews_ints[:10]

[[78, 57, 6612, 218],
 [78, 540, 541, 1122, 2453, 1, 2, 197, 6613],
 [78, 3, 185, 98, 775, 563, 3, 75, 1, 148, 143, 191],
 [78,
  64,
  132,
  3663,
  1,
  4598,
  4599,
  981,
  15,
  21,
  3091,
  3664,
  61,
  54,
  22,
  487,
  2714],
 [78, 10, 64, 4, 132, 469, 207, 488, 81, 20],
 [78,
  429,
  76,
  275,
  291,
  4,
  8,
  7,
  192,
  26,
  185,
  22,
  29,
  2051,
  64,
  132,
  2,
  112,
  207,
  488,
  81,
  125,
  1781],
 [78, 167, 1427, 301, 47, 3, 106, 2221, 29, 4600, 6614, 211, 128, 464],
 [78, 132, 280, 4, 3092, 1690, 7, 3665, 325, 3666, 6615, 71, 6616],
 [78, 187, 3, 6617, 39, 3, 43, 1592],
 [78, 20, 23, 349, 10, 564, 40, 91, 357, 227, 168, 133, 1, 19]]

+ 虽然使用整值索引替代了评论中的word，但是由于评论的word数量不一致，有短有长，所以设置`seq_len=30`，保证每个长度都是30，不够补零，多的取前30个，我们使用keras中的`preprocessing`模块可以很方便的完成这个任务

In [12]:
seq_len = 30
from tensorflow.contrib.keras import preprocessing
features = np.zeros((len(reviews_ints),seq_len),dtype=int)
#将reviews_ints值逐行 赋值给features
features = preprocessing.sequence.pad_sequences(reviews_ints, seq_len)
features.shape

(14640, 30)

In [13]:
features

array([[    0,     0,     0, ...,    57,  6612,   218],
       [    0,     0,     0, ...,     2,   197,  6613],
       [    0,     0,     0, ...,   148,   143,   191],
       ...,
       [    0,     0,     0, ...,   241,     1, 16722],
       [    0,     0,     0, ...,   126,    11,  2620],
       [   13,    38,    22, ...,     2,   173,     8]], dtype=int32)

In [14]:
# lables
Y = pd.get_dummies(data['airline_sentiment']).values
encoded_labels = Y

# data拆分比例
split_frac = 0.8

# 将data拆分成训练、验证、测试三部分，包括整值索引与标签
split_idx = int(len(features)*split_frac)
train_x, remaining_x = features[:split_idx], features[split_idx:]
train_y, remaining_y = encoded_labels[:split_idx], encoded_labels[split_idx:]

# 其中验证与测试比例相同
test_idx = int(len(remaining_x)*0.5)
val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]
 
# 输出训练、验证、测试集的大小
print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

			Feature Shapes:
Train set: 		(11712, 30) 
Validation set: 	(1464, 30) 
Test set: 		(1464, 30)


In [0]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# 将训练、验证、测试集转换成Tensor
train_data = TensorDataset(torch.from_numpy(np.array(train_x)), torch.from_numpy(np.array(train_y)))
valid_data = TensorDataset(torch.from_numpy(np.array(val_x)), torch.from_numpy(np.array(val_y)))
test_data = TensorDataset(torch.from_numpy(np.array(test_x)), torch.from_numpy(np.array(test_y)))
 
batch_size = 50

# 训练、验证、测试集 Dataloader
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [16]:
# 展示一批训练数据
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()
 
print('Sample input size: ', sample_x.size()) # batch_size=50, seq_len=30
print('Sample input: \n', sample_x)

print('Sample label size: ', sample_y.size()) # batch_size=50
print('Sample label: \n', sample_y)

Sample input size:  torch.Size([50, 30])
Sample input: 
 tensor([[    0,     0,     0,  ...,   422,  3586,   693],
        [    0,     0,     0,  ...,    29,    14,  1543],
        [    0,     0,     0,  ...,  1044,    29,   936],
        ...,
        [    0,     0,     0,  ...,   167, 14490,    14],
        [    0,     0,     0,  ...,     9,   249, 11762],
        [    0,     0,     0,  ...,  7728,   420,  3287]], dtype=torch.int32)
Sample label size:  torch.Size([50, 3])
Sample label: 
 tensor([[0, 0, 1],
        [0, 0, 1],
        [1, 0, 0],
        [0, 0, 1],
        [0, 1, 0],
        [0, 1, 0],
        [0, 0, 1],
        [0, 1, 0],
        [0, 0, 1],
        [0, 0, 1],
        [0, 1, 0],
        [0, 0, 1],
        [0, 0, 1],
        [0, 1, 0],
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 1],
        [0, 0, 1],
        [0, 1, 0],
        [0, 0, 1],
        [0, 1, 0],
        [0, 0, 1],
        [0, 0, 1],
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 1],
        [1, 

In [17]:
# 检查有没有GPU
train_on_gpu=torch.cuda.is_available()
 
if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')

Training on GPU.


使用pytorch搭建RNN网络,来完成情感分析的任务

In [0]:
import torch.nn as nn
 
class SentimentRNN(nn.Module):
 
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, bidirectional=True, drop_prob=0.5):
       
        super(SentimentRNN, self).__init__()
 
#         self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        # 双向RNN
        self.bidirectional = bidirectional
        
        # 进行Embedding操作
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        # LSTM层
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True,
                            bidirectional=bidirectional)
        
        # dropout层
        self.dropout = nn.Dropout(0.3)
        
        # 是否使用双向RNN
        if bidirectional:
          self.fc = nn.Linear(hidden_dim*2, output_size)
        else:
          self.fc = nn.Linear(hidden_dim, output_size)
         
        # softmax层
        self.soft = nn.Softmax()
        
 
    def forward(self, x, hidden):
       
        batch_size = x.size(0)
 
        x = x.long()
        # Embedding层的输出
        embeds = self.embedding(x)
        # LSTM层的输出
        lstm_out, hidden = self.lstm(embeds, hidden)
        
#         if bidirectional:
#           lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim*2)
#         else:
#           lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
       
        # dropout与fully-connected层
#         out = self.dropout(lstm_out)
        out = lstm_out
        out = out[:, -1]
        out = self.fc(out)
        # softmax 
#         soft_out = out
        soft_out = self.soft(out)
        
        # reshape to be batch_size first
#         soft_out = soft_out.view(batch_size, -1)
#         soft_out = soft_out[:, -1] # get last batch of labels
#         soft_out = np.where(soft_out==np.max(soft_out))
    
        return soft_out, hidden
    
    
    def init_hidden(self, batch_size):

        weight = next(self.parameters()).data
        
        number = 1
        if self.bidirectional:
           number = 2
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers*number, batch_size, self.hidden_dim).zero_().cuda(),
                      weight.new(self.n_layers*number, batch_size, self.hidden_dim).zero_().cuda()
                     )
        else:
            hidden = (weight.new(self.n_layers*number, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers*number, batch_size, self.hidden_dim).zero_()
                     )
        
        return hidden

In [28]:
# 设置超参数
vocab_size = len(vocab_to_int)+1 # +1 for the 0 padding + our word tokens
output_size = 3
embedding_dim = 128
hidden_dim = 196
n_layers = 2
bidirectional = False  #这里若为True，则是双向LSTM
 
net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, bidirectional)

print(net)

SentimentRNN(
  (embedding): Embedding(16723, 128)
  (lstm): LSTM(128, 196, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3)
  (fc): Linear(in_features=196, out_features=3, bias=True)
  (soft): Softmax()
)


In [34]:
output.squeeze()

tensor([[0.3335, 0.3447, 0.3218],
        [0.3288, 0.3425, 0.3287],
        [0.3234, 0.3482, 0.3284],
        [0.3141, 0.3554, 0.3305],
        [0.3210, 0.3442, 0.3348],
        [0.3272, 0.3474, 0.3254],
        [0.3171, 0.3506, 0.3323],
        [0.3289, 0.3394, 0.3318],
        [0.3260, 0.3419, 0.3321],
        [0.3249, 0.3491, 0.3261],
        [0.3175, 0.3451, 0.3373],
        [0.3280, 0.3408, 0.3312],
        [0.3269, 0.3457, 0.3275],
        [0.3221, 0.3436, 0.3344],
        [0.3242, 0.3465, 0.3293],
        [0.3256, 0.3501, 0.3242],
        [0.3203, 0.3422, 0.3376],
        [0.3189, 0.3465, 0.3346],
        [0.3205, 0.3464, 0.3331],
        [0.3290, 0.3404, 0.3305],
        [0.3255, 0.3429, 0.3317],
        [0.3177, 0.3582, 0.3241],
        [0.3191, 0.3514, 0.3294],
        [0.3262, 0.3409, 0.3330],
        [0.3252, 0.3509, 0.3239],
        [0.3155, 0.3531, 0.3314],
        [0.3238, 0.3513, 0.3249],
        [0.3228, 0.3572, 0.3199],
        [0.3234, 0.3381, 0.3385],
        [0.328

In [46]:
output.long()

tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]], device='cuda:0')

In [42]:
labels.long().float()

tensor([[1., 0., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [1., 0., 0.],
        [1., 0., 0.],
        [0

开始训练：

In [43]:
# 使用交叉熵损失，设置学习率为0.001，使用Adam优化算法
lr=0.001
 
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
 
# 训练4轮 
epochs = 4 
# 每100步做一次输出 
print_every = 100
# 为了进行梯度裁剪，防止反向传播过程中出现梯度消失或者爆炸的情况
clip=5 
 
# 使用GPU
if(train_on_gpu):
    net.cuda()

# 训练
net.train()
for e in range(epochs):
    
    # 隐层
    h = net.init_hidden(batch_size)
    counter = 0
 
    # 按批量遍历
    for inputs, labels in train_loader:
        counter += 1
 
        if(train_on_gpu):
            inputs, labels = inputs.cuda(), labels.cuda()
 
        h = tuple([each.data for each in h])
        # 权值初始化
        net.zero_grad()
        # 得到模型输出
        output, h = net(inputs, h)
        
        # 损失值
        loss = criterion(output.squeeze(), labels.long().float())
        # 反向传播
        loss.backward()
        # 做一个梯度裁剪
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        # 更新权值
        optimizer.step()
 
        if counter % print_every == 0:
            # 获得验证集loss，与训练集过程类似
            val_h = net.init_hidden(batch_size)
            val_losses = []
            net.eval()
            for inputs, labels in valid_loader:
 
                val_h = tuple([each.data for each in val_h])
 
                if(train_on_gpu):
                    inputs, labels = inputs.cuda(), labels.cuda()
 
                output, val_h = net(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels.long().float())
 
                val_losses.append(val_loss.item())
 
            net.train()
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "训练损失: {:.6f}...".format(loss.item()),
                  "验证损失: {:.6f}".format(np.mean(val_losses)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:57: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


RuntimeError: ignored

开始测试：

In [0]:
test_losses = [] 
num_correct = 0
 
# 隐层
h = net.init_hidden(batch_size)
 
net.eval()
# 将测试数据进行迭代遍历
for inputs, labels in test_loader:
 
    h = tuple([each.data for each in h])
 
    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
    
    # 得到预测输出
    output, h = net(inputs, h)
    
    # 测试损失
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())
    
#     pred = torch.round(output.squeeze())  
    
    # compare predictions to true label
#     correct_tensor = pred.eq(labels.float().view_as(pred))
#     correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
#     num_correct += np.sum(correct)
 
 
# avg test loss
print("Test loss: {:.3f}".format(np.mean(test_losses)))
 
# accuracy over all test data
# test_acc = num_correct/len(test_loader.dataset)
# print("Test accuracy: {:.3f}".format(test_acc))f